In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# PDF 파싱을 위한 필수 라이브러리들을 임포트합니다
import os
from layoutparse.utils import (
    SplitPDFFilesNode,
)  # PDF 파일을 분할하는 노드를 임포트합니다
from layoutparse.state import ParseState  # 파싱 상태를 관리하는 클래스를 임포트합니다
from layoutparse.upstage import (  # Upstage 관련 주요 노드들을 임포트합니다
    DocumentParseNode,  # 문서 파싱을 담당하는 노드
    PostDocumentParseNode,  # 파싱 후처리를 담당하는 노드
    WorkingQueueNode,  # 작업 큐를 관리하는 노드
    continue_parse,  # 파싱 계속 여부를 결정하는 함수
)
from langgraph.graph import StateGraph  # 그래프 상태를 관리하는 클래스를 임포트합니다
from langgraph.checkpoint.memory import (
    MemorySaver,
)  # 메모리 체크포인트 저장을 위한 클래스입니다
from langchain_teddynote.graphs import (
    visualize_graph,
)  # 그래프 시각화를 위한 함수를 임포트합니다


# PDF 파일을 30페이지 단위로 분할하는 노드를 생성합니다
# verbose=True로 설정하면 노드의 실행 상태를 출력합니다
split_pdf_node = SplitPDFFilesNode(batch_size=30, test_page=None, verbose=True)

# Upstage API를 사용하여 문서를 파싱하는 노드를 생성합니다
document_parse_node = DocumentParseNode(
    api_key=os.environ["UPSTAGE_API_KEY"],use_ocr=True, verbose=True
)

# 파싱 후처리를 담당하는 노드를 생성합니다
post_document_parse_node = PostDocumentParseNode(verbose=True)
# 작업 큐를 관리하는 노드를 생성합니다
working_queue_node = WorkingQueueNode(verbose=True)


# ParseState를 기반으로 하는 StateGraph 워크플로우를 생성합니다
workflow = StateGraph(ParseState)

# 워크플로우에 각각의 노드들을 추가합니다
workflow.add_node("split_pdf_node", split_pdf_node)
workflow.add_node("document_parse_node", document_parse_node)
workflow.add_node("post_document_parse_node", post_document_parse_node)
workflow.add_node("working_queue_node", working_queue_node)

# 노드들 간의 연결 관계를 설정합니다
workflow.add_edge("split_pdf_node", "working_queue_node")
# 작업 큐 노드에서 조건에 따라 다음 노드를 결정하는 분기를 추가합니다
workflow.add_conditional_edges(
    "working_queue_node",
    continue_parse,
    {True: "document_parse_node", False: "post_document_parse_node"},
)
workflow.add_edge("document_parse_node", "working_queue_node")

# 워크플로우의 시작점을 PDF 분할 노드로 설정합니다
workflow.set_entry_point("split_pdf_node")

# 최종적으로 메모리 체크포인터를 사용하여 워크플로우를 컴파일합니다
document_parse_graph = workflow.compile(checkpointer=MemorySaver())

In [ ]:
# 그래프를 시각화합니다
visualize_graph(document_parse_graph)

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph
import uuid

"""
RunnableConfig를 사용하여 그래프 실행에 필요한 설정을 정의합니다.
recursion_limit: 재귀 호출의 제한을 설정하여 무한 루프를 방지합니다
configurable: 실행 시 필요한 설정값들을 정의합니다
"""
config = RunnableConfig(
    recursion_limit=300,  # 재귀 호출 제한을 300으로 설정합니다
    configurable={"thread_id": str(uuid.uuid4())},
)

# ParseState 객체를 생성하여 PDF 파일 처리를 위한 초기 상태를 설정합니다
inputs = ParseState(
    filepath="data/test_data.pdf",  # 처리할 PDF 파일의 경로를 지정합니다
    language="English",  # 문서의 언어를 영어로 설정합니다
)

# 그래프를 스트리밍 모드로 실행하여 실시간으로 처리 상태를 확인합니다
stream_graph(
    document_parse_graph,
    inputs,
    config=config,
)

In [ ]:
# 실행 결과
snapshot = document_parse_graph.get_state(config).values

In [ ]:
snapshot["elements_from_parser"][:2]

In [ ]:
from layoutparse.teddynote_parser import create_upstage_parser_graph

# 그래프 생성
upstage_parser_graph = create_upstage_parser_graph(
    batch_size=30, test_page=None, verbose=True
)

In [ ]:
import uuid
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph

# 옵션 설정
config = RunnableConfig(
    recursion_limit=300,
    configurable={"thread_id": str(uuid.uuid4())},
)

# filepath: 분석할 PDF 파일의 경로
inputs = {
    "filepath": "data/argus-bitumen.pdf",
}

stream_graph(upstage_parser_graph, inputs, config=config)